In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [3]:
first_np, second_np, label_np  = data.getData()
first_np, second_np = utils.process_splitted(first_np, second_np)
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

linearized complete!
done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [4]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 133 pretrained vectors found.


In [5]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [6]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [7]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('cnn-ocr/tests')

In [8]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [9]:
# ====================
# set parameters here
# ====================

title = 'cnn-test10'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 200
hidden_size = 100
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.2

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=pretrained_word2vec
freeze_embedding=False,

In [10]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!
OCR_cnn(
  (emb): Embedding(133, 200)
  (cnn1): CNN(
    (conv1d_list): ModuleList(
      (0): Conv1d(200, 100, kernel_size=(15,), stride=(1,))
      (1): Conv1d(200, 200, kernel_size=(21,), stride=(1,))
      (2): Conv1d(200, 100, kernel_size=(15,), stride=(1,))
    )
  )
  (cnn2): CNN(
    (conv1d_list): ModuleList(
      (0): Conv1d(200, 100, kernel_size=(15,), stride=(1,))
      (1): Conv1d(200, 200, kernel_size=(21,), stride=(1,))
      (2): Conv1d(200, 100, kernel_size=(15,), stride=(1,))
    )
  )
  (fc1): Linear(in_features=800, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [11]:
trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.535848   | 71.757202  | 0.373186 | 83.19  | 19.73 
   2    |   0.370221   | 82.193416  | 0.298440 | 86.74  | 19.29 
   3    |   0.322161   | 84.995885  | 0.293312 | 86.93  | 19.28 
   4    |   0.296760   | 86.358025  | 0.264258 | 88.74  | 19.29 
   5    |   0.286257   | 86.905350  | 0.258040 | 88.74  | 19.29 
   6    |   0.271404   | 87.769547  | 0.252147 | 88.41  | 19.40 
   7    |   0.254484   | 88.576132  | 0.248873 | 89.89  | 19.49 
   8    |   0.250893   | 88.979424  | 0.225216 | 90.48  | 19.28 
   9    |   0.234286   | 89.823045  | 0.251214 | 89.56  | 19.35 
  10    |   0.236620   | 89.744856  | 0.232611 | 90.44  | 19.29 
  11    |   0.223621   | 90.559671  | 0.271627 | 88.93  | 19.29 
  12    |   0.221485   | 90.600823  | 0.229682 | 90.85  | 19.31 
  13    |   0.212018   | 90.934156  | 0.223947 | 90.7

In [22]:
model = mu.getModel('cnn-test10')
print(model)

OCR_cnn(
  (emb): Embedding(133, 200)
  (cnn1): CNN(
    (conv1d_list): ModuleList(
      (0): Conv1d(200, 100, kernel_size=(15,), stride=(1,))
      (1): Conv1d(200, 200, kernel_size=(21,), stride=(1,))
      (2): Conv1d(200, 100, kernel_size=(14,), stride=(1,))
    )
  )
  (cnn2): CNN(
    (conv1d_list): ModuleList(
      (0): Conv1d(200, 100, kernel_size=(15,), stride=(1,))
      (1): Conv1d(200, 200, kernel_size=(21,), stride=(1,))
      (2): Conv1d(200, 100, kernel_size=(14,), stride=(1,))
    )
  )
  (fc1): Linear(in_features=800, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [23]:
tester.test(test_dataloader=test_dataloader,
            device=device,
            model=model,
            title=title)

test loss:  0.17445512680957714
test acc:  93.53333333333333
saved precision and recall results to file!


(tensor([0., 1., 1.,  ..., 1., 0., 1.], device='cuda:0'),
 tensor([0., 1., 0.,  ..., 1., 0., 1.], device='cuda:0'))

In [14]:
# mu.graphModel(train_dataloader, model, writer)

In [15]:
mu.saveModel(title, model)